In [1]:
import pandas as pd
from datetime import datetime as dt

import integrate as intr



In [2]:

#miv_df = intr.process_miv_data()
#fb_data = intr.process_foot_bike_data()

In [3]:
#uplicate_rows = miv_df[miv_df.duplicated()]
#print(duplicate_rows.shape[0])

In [17]:
acc_df = intr.process_accident_data(True)


u_string RoadTrafficAccidentLocations.json
Filepath: datasets/RoadTrafficAccidentLocations.json


2024-01-03 15:17:14,309 - integrate.py - DEBUG -   AccidentInvolvingPedestrian AccidentInvolvingBicycle  \
0                       false                    false   
1                       false                     true   
2                       false                    false   
3                       false                    false   
4                       false                    false   

  AccidentInvolvingMotorcycle  
0                       false  
1                       false  
2                       false  
3                       false  
4                       false  


,AccidentUID,AccidentYear,AccidentMonth,AccidentWeekDay_en,AccidentHour,NKoord,EKoord,AccidentType_en,AccidentType,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,RoadType_en,geometry


In [26]:
#acc_df[acc_df['AccidentInvolvingBicycle']==True]
acc_df.head()

,AccidentUID,AccidentYear,AccidentMonth,AccidentWeekDay_en,AccidentHour,NKoord,EKoord,AccidentType_en,AccidentType,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,RoadType_en,geometry
0,A2D2677533867004E0430A865E337004,2011,1,Saturday,0,1245194,2684605,Accident with skidding or self-accident,at0,as4,True,True,True,rt433,Minor road,POINT Z (8.55841 47.352168 0)
1,9FD6441F802C20A6E0430A865E3320A6,2011,1,Saturday,1,1246980,2682382,Accident with skidding or self-accident,at0,as3,True,True,True,rt433,Minor road,POINT Z (8.52932 47.368512 0)
2,9FDA0DC4856A6094E0430A865E336094,2011,1,Saturday,2,1247749,2682791,Accident with skidding or self-accident,at0,as4,True,True,True,rt439,Other,POINT Z (8.534877 47.375376 0)
3,A3B66E42396E6000E0430A865E336000,2011,1,Saturday,2,1247102,2681199,Accident when crossing the lane(s),at5,as3,True,True,True,rt433,Minor road,POINT Z (8.513682 47.369756 0)
4,9FDA0DBE8CCE9096E0430A865E339096,2011,1,Saturday,3,1250690,2682479,Accident with skidding or self-accident,at0,as4,True,True,True,rt433,Minor road,POINT Z (8.531288 47.401865 0)


In [5]:
acc_df.head()
acc_df['AccidentWeekDay_en'].unique()
#acc_df.dtypes



array(['Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday',
       'Friday'], dtype=object)

In [6]:
print("Accident Columns:")
print(acc_df.dtypes)


Accident Columns:
AccidentUID                    object
AccidentYear                    int64
AccidentMonth                   int64
AccidentWeekDay_en             object
AccidentHour                    int64
NKoord                          int64
EKoord                          int64
AccidentType_en                object
AccidentType                   object
AccidentSeverityCategory       object
AccidentInvolvingPedestrian      bool
AccidentInvolvingBicycle         bool
AccidentInvolvingMotorcycle      bool
RoadType                       object
RoadType_en                    object
geometry                       object
dtype: object


In [7]:
acc_df['ID'] = acc_df.index +1
acc_df[('ID')]

0            1
1            2
2            3
3            4
4            5
         ...  
55821    55822
55822    55823
55823    55824
55824    55825
55825    55826
Name: ID, Length: 55826, dtype: int64

,AccidentUID,AccidentYear,AccidentMonth,AccidentWeekDay_en,AccidentHour,NKoord,EKoord,AccidentType_en,AccidentType,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,RoadType_en,geometry,ID


In [9]:
acc_df.head()

,AccidentUID,AccidentYear,AccidentMonth,AccidentWeekDay_en,AccidentHour,NKoord,EKoord,AccidentType_en,AccidentType,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,RoadType_en,geometry,ID
0,A2D2677533867004E0430A865E337004,2011,1,Saturday,0,1245194,2684605,Accident with skidding or self-accident,at0,as4,True,True,True,rt433,Minor road,POINT Z (8.55841 47.352168 0),1
1,9FD6441F802C20A6E0430A865E3320A6,2011,1,Saturday,1,1246980,2682382,Accident with skidding or self-accident,at0,as3,True,True,True,rt433,Minor road,POINT Z (8.52932 47.368512 0),2
2,9FDA0DC4856A6094E0430A865E336094,2011,1,Saturday,2,1247749,2682791,Accident with skidding or self-accident,at0,as4,True,True,True,rt439,Other,POINT Z (8.534877 47.375376 0),3
3,A3B66E42396E6000E0430A865E336000,2011,1,Saturday,2,1247102,2681199,Accident when crossing the lane(s),at5,as3,True,True,True,rt433,Minor road,POINT Z (8.513682 47.369756 0),4
4,9FDA0DBE8CCE9096E0430A865E339096,2011,1,Saturday,3,1250690,2682479,Accident with skidding or self-accident,at0,as4,True,True,True,rt433,Minor road,POINT Z (8.531288 47.401865 0),5


In [10]:
from sqlalchemy import create_engine
from geoalchemy2 import Geometry, WKTElement
import geopandas as gpd
from shapely import wkt

db_url = f'postgresql://seb:@localhost:5432/test-db23'
engine =  create_engine(db_url)

#miv_df.to_sql('table_name', engine, if_exists='replace', index=False)
#fb_data.to_sql('footbike', engine, if_exists='replace', index=False)

geometry_column = 'geometry'


acc_df['geometry'] = acc_df['geometry'].apply(lambda geom: WKTElement(geom.wkt, srid=4326))

acc_df.to_sql('accidents', engine, if_exists='replace', index=False, dtype={'geometry': Geometry('POINT', srid=4326)})



AttributeError: 'str' object has no attribute 'wkt'

In [ ]:
engine.dispose()